In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install monai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00


In [3]:
!pip install torchmetrics==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 6.4 MB/s eta 0:00:00


In [4]:
!pip install pytorch-lightning
!pip install transformers
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00


In [5]:
!pip install tdqm

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=ba10273f22430016fcad6b83799386fae6d9e57b268f615ca08db527e0249a69
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


In [6]:
!pip install timm==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 6.3 MB/s eta 0:00:00


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random
import py
from sklearn.metrics import classification_report
from monai.data import decollate_batch, DataLoader, CacheDataset, ThreadDataLoader
from monai.metrics import ROCAUCMetric
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    Compose,
    LoadImaged,
    ScaleIntensityd,
    EnsureTyped,
    Resized,
    CropForegroundd,
    SpatialPadd,
    CastToTyped,
    ConcatItemsd,
)
from monai.utils import set_determinism
import argparse
import math

In [8]:
import sys
import os
import requests

import torch
import numpy as np

import matplotlib.pyplot as plt
from PIL import Image

# check whether run in Colab
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install timm==0.4.5  # 0.3.2 does not work in Colab
    !git clone https://github.com/facebookresearch/mae.git
    sys.path.append('./mae')
else:
    sys.path.append('..')
import models_mae

Running in Colab.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 0.4.12
    Uninstalling timm-0.4.12:
      Successfully uninstalled timm-0.4.12
Cloning into 'mae'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Receiving objects: 100% (39/39), 829.54 KiB | 673.00 KiB/s, done.
Resolving deltas: 100% (12/12), done.


In [9]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def show_image(image, title=''):
    # image is [H, W, 3]
    assert image.shape[2] == 3
    plt.imshow(torch.clip((image * imagenet_std + imagenet_mean) * 255, 0, 255).int())
    plt.title(title, fontsize=16)
    plt.axis('off')
    return

def prepare_model(chkpt_dir, arch='mae_vit_large_patch16'):
    # build model
    model = getattr(models_mae, arch)()
    # load model
    checkpoint = torch.load(chkpt_dir, map_location='cpu')
    msg = model.load_state_dict(checkpoint['model'], strict=False)
    print(msg)
    return model

def run_one_image(img, model):
    x = torch.tensor(img)

    # make it a batch-like
    x = x.unsqueeze(dim=0)
    x = torch.einsum('nhwc->nchw', x)

    # run MAE
    loss, y, mask = model(x.float(), mask_ratio=0.75)
    y = model.unpatchify(y)
    y = torch.einsum('nchw->nhwc', y).detach().cpu()

    # visualize the mask
    mask = mask.detach()
    mask = mask.unsqueeze(-1).repeat(1, 1, model.patch_embed.patch_size[0]**2 *3)  # (N, H*W, p*p*3)
    mask = model.unpatchify(mask)  # 1 is removing, 0 is keeping
    mask = torch.einsum('nchw->nhwc', mask).detach().cpu()

    x = torch.einsum('nchw->nhwc', x)

    # masked image
    im_masked = x * (1 - mask)

    # MAE reconstruction pasted with visible patches
    im_paste = x * (1 - mask) + y * mask

    # make the plt figure larger
    plt.rcParams['figure.figsize'] = [24, 24]

    plt.subplot(1, 4, 1)
    show_image(x[0], "original")

    plt.subplot(1, 4, 2)
    show_image(im_masked[0], "masked")

    plt.subplot(1, 4, 3)
    show_image(y[0], "reconstruction")

    plt.subplot(1, 4, 4)
    show_image(im_paste[0], "reconstruction + visible")

    plt.show()

In [10]:
def set_seed(no):
    torch.manual_seed(no)
    random.seed(no)
    np.random.seed(no)
    os.environ['PYTHONHASHSEED'] = str()
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(100)

In [25]:
import pandas as pd
import os
from enum import Enum
from monai.transforms import Transform

from numpy.core.arrayprint import printoptions


class Modality(Enum):
    MR_T1 = 1
    MR_T2 = 2
    MR_T2_STAR = 3
    MR_FLAIR = 4
    MR_TOF_MRA = 5

# def create_oasis_3_unimodal_dataset(csv_path, dataset_root, modality, transform,
#                                     cache_rate):
#     train_df = pd.read_csv(csv_path, sep=";")
#     train_df.fillna('', inplace=True)

#     column_name = ""
#     if modality == Modality.MR_T1:
#         column_name = "MR T1w"
#     elif modality == Modality.MR_T2:
#         column_name = "MR T2w"
#     elif modality == Modality.MR_T2_STAR:
#         column_name = "MR T2*"
#     elif modality == Modality.MR_FLAIR:
#         column_name = "MR FLAIR"
#     elif modality == Modality.MR_TOF_MRA:
#         column_name = "MR TOF-MRA"
#     else:
#         assert (False)

#     train_data = []
#     for index, row in train_df.iterrows():
#         rel_path = row[column_name]
#         if not rel_path:
#             continue
#         image_path = os.path.join(dataset_root, rel_path)
#         train_data.append({"image": image_path, "label": row["label"]})

#     # for root, dirs, files in os.walk(dataset_root):
#     #     for file_name in files:
#     #         file_path = os.path.join(root, file_name)
#     #         train_data.append({"image": file_path, "label":0})

#     return CacheDataset(data=train_data, transform=transform, cache_rate=cache_rate, num_workers=5,
#                         copy_cache=False)
modality_names = ["MR T1w", "MR T2w", "MR T2*", "MR FLAIR", "MR TOF-MRA"]

def create_oasis_3_multimodal_dataset(csv_path: str, dataset_root: str, transform: Transform, cache_rate: float, missing_modality: str):
    train_df = pd.read_csv(csv_path, sep=";")
    train_df.fillna('', inplace=True)

    train_data = []
    for index, row in train_df.iterrows():
        data_dict = {}
        has_non_empty = False
        for modality in modality_names:
            file_path = row[modality]
            if file_path:
                has_non_empty = True
                data_dict[modality] = os.path.join(dataset_root, file_path)
            else:
                if missing_modality == "zeros":
                    data_dict[modality] = "/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS/empty_volume_2d.nii.gz"
                elif missing_modality == "gauss":
                    data_dict[modality] = "/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS/gauss_2d_256.nii.gz"
                else:
                    raise ValueError(f"Invalid missing modality key {missing_modality}")
        if not has_non_empty:
            continue
        data_dict["label"] = row["label"]
        train_data.append(data_dict)
    print(train_data)
    return CacheDataset(data=train_data, transform=transform, cache_rate=cache_rate, num_workers=5, copy_cache=False)

class SafeCropForegroundd:
    def __init__(self, keys, source_key, select_fn, margin=0):
        self.source_key = source_key
        self.crop_foreground = CropForegroundd(keys=keys, source_key=source_key, select_fn=select_fn, margin=margin)

    def __call__(self, data):
        cropped_data = self.crop_foreground(data.copy())
        cropped_image = cropped_data[self.source_key]

        # Check if any dimension (excluding batch and channel dimensions) is zero.
        if np.any(np.asarray(cropped_image.shape[1:]) == 0):
            return data  # Revert to original data if cropped size is zero in any dimension

        return cropped_data

In [12]:
!pip freeze > requirements.txt

In [26]:
dataset_root = r"/content/drive/MyDrive/OASIS-3-MR-Sessions-2D"

In [27]:
resolution = 224
cache_rate = 0.1
batch_size = 16
dataset_root = r"/content/drive/MyDrive/OASIS-3-MR-Sessions-2D"
missing_modality = "gauss"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
foreground_crop_threshold = 0.1
transform_list = [
        LoadImaged(keys=modality_names, image_only=True),
        EnsureChannelFirstd(keys=modality_names + ["label"], channel_dim="no_channel"),
        CastToTyped("label", dtype=np.float64),
        ScaleIntensityd(keys=modality_names),
    ]
for i in range(len(modality_names)):
     transform_list.append(
         SafeCropForegroundd(keys=modality_names[i], source_key=modality_names[i], select_fn=lambda x: x > foreground_crop_threshold, margin=5)
     )
transform_list.extend([
    Resized(keys=modality_names, spatial_size=resolution, size_mode="longest"),
    SpatialPadd(keys=modality_names, spatial_size=(resolution, resolution)),
    ConcatItemsd(keys=modality_names, name="image"),
    EnsureTyped(keys=["image"], device=device),
]
)
transform = Compose(transform_list)

train_table_path = r"/content/drive/MyDrive/MLMI/oasis_3_multimodal_train.csv"
train_ds = create_oasis_3_multimodal_dataset(csv_path=train_table_path, dataset_root=dataset_root, transform=transform, cache_rate=cache_rate, missing_modality=missing_modality)
train_loader = ThreadDataLoader(train_ds, num_workers=0, batch_size=batch_size, shuffle=True)

val_table_path = r"/content/drive/MyDrive/MLMI/oasis_3_multimodal_val_all.csv"
val_ds = create_oasis_3_multimodal_dataset(csv_path=val_table_path, dataset_root=dataset_root, transform=transform, cache_rate=cache_rate, missing_modality=missing_modality)
val_loader = ThreadDataLoader(val_ds, num_workers=0, batch_size=batch_size, shuffle=True)



[{'MR T1w': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS31022_MR_d0089/anat2/NIFTI/sub-OAS31022_ses-d0089_T1w.nii.gz', 'MR T2w': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS/gauss_2d_256.nii.gz', 'MR T2*': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS31022_MR_d0089/anat3/NIFTI/sub-OAS31022_ses-d0089_T2star.nii.gz', 'MR FLAIR': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS31022_MR_d0089/anat1/NIFTI/sub-OAS31022_ses-d0089_FLAIR.nii.gz', 'MR TOF-MRA': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS/gauss_2d_256.nii.gz', 'label': 0}, {'MR T1w': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS31114_MR_d2658/anat5/NIFTI/sub-OAS31114_ses-d2658_run-02_T1w.nii.gz', 'MR T2w': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS31114_MR_d2658/anat7/NIFTI/sub-OAS31114_ses-d2658_T2w.nii.gz', 'MR T2*': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS31114_MR_d2658/anat6/NIFTI/sub-OAS31114_ses-d2658_T2star.nii.gz', 'MR FLAIR': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/

Loading dataset: 100%|██████████| 255/255 [00:06<00:00, 38.59it/s]


[{'MR T1w': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS30419_MR_d0621/anat3/NIFTI/sub-OAS30419_ses-d0621_run-02_T1w.nii.gz', 'MR T2w': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS30419_MR_d0621/anat4/NIFTI/sub-OAS30419_ses-d0621_T2w.nii.gz', 'MR T2*': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS/gauss_2d_256.nii.gz', 'MR FLAIR': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS/gauss_2d_256.nii.gz', 'MR TOF-MRA': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS/gauss_2d_256.nii.gz', 'label': 0}, {'MR T1w': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS30671_MR_d3613/anat2/NIFTI/sub-OAS30671_ses-d3613_T1w.nii.gz', 'MR T2w': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS/gauss_2d_256.nii.gz', 'MR T2*': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS30671_MR_d3613/anat3/NIFTI/sub-OAS30671_ses-d3613_T2star.nii.gz', 'MR FLAIR': '/content/drive/MyDrive/OASIS-3-MR-Sessions-2D/OAS30671_MR_d3613/anat1/NIFTI/sub-OAS30671_ses-d3613_FLAIR.nii.gz', 'MR TOF-MRA': '/c

Loading dataset: 100%|██████████| 28/28 [00:00<00:00, 41.10it/s]


In [28]:
# This is an MAE model trained with pixels as targets for visualization (ViT-Large, training mask ratio=0.75)

# download checkpoint if not exist
!wget -nc https://dl.fbaipublicfiles.com/mae/visualize/mae_visualize_vit_base.pth

chkpt_dir = 'mae_visualize_vit_base.pth'
mae = prepare_model(chkpt_dir, 'mae_vit_base_patch16')
print('Model loaded.')
print(mae(torch.ones([3,3,224,224])))

File ‘mae_visualize_vit_base.pth’ already there; not retrieving.

<All keys matched successfully>
Model loaded.
(tensor(2.9613e-05, grad_fn=<DivBackward0>), tensor([[[0.6598, 0.6760, 0.6741,  ..., 0.6733, 0.6513, 0.7137],
         [0.9913, 0.9910, 0.9956,  ..., 1.0022, 1.0017, 1.0078],
         [0.9981, 0.9968, 0.9972,  ..., 0.9979, 1.0017, 1.0024],
         ...,
         [0.6780, 0.6784, 0.6808,  ..., 0.6664, 0.6796, 0.7322],
         [0.9991, 0.9972, 1.0083,  ..., 1.0103, 1.0137, 1.0219],
         [0.9964, 0.9933, 1.0040,  ..., 0.9992, 1.0035, 1.0077]],

        [[1.0049, 1.0026, 0.9991,  ..., 1.0032, 1.0036, 1.0064],
         [0.9983, 0.9935, 0.9985,  ..., 1.0048, 1.0016, 1.0093],
         [0.6787, 0.6871, 0.6675,  ..., 0.6580, 0.6456, 0.7104],
         ...,
         [0.9970, 0.9990, 1.0110,  ..., 1.0020, 1.0081, 1.0130],
         [1.0013, 1.0013, 1.0111,  ..., 0.9992, 1.0047, 1.0116],
         [0.9995, 1.0052, 1.0130,  ..., 0.9930, 0.9963, 0.9941]],

        [[1.0018, 0.9997, 1.004

In [29]:
class MAE(torch.nn.Module):
    def __init__(self, mae, num_classes=1) -> None:
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(len(modality_names), 3, kernel_size=1, stride=1, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(3)
        )
        self.mae=mae




    def forward(self, img,mask_ratio=0.75):
        img = self.conv(img)
        latent, mask, ids_restore = self.mae.forward_encoder(img, mask_ratio)
        pred = self.mae.forward_decoder(latent, ids_restore)
          # [N, L, p*p*3]
        loss = self.mae.forward_loss(img, pred, mask)
        return loss, pred, mask
model = MAE(mae.cuda()).cuda()

In [30]:
optim = torch.optim.AdamW(model.parameters(), lr=0.003 * 12 / 256, betas=(0.9, 0.95), weight_decay=0.05)
lr_func = lambda epoch: min((epoch + 1) / (10 + 1e-8), 0.5 * (math.cos(epoch / 150 * math.pi) + 1))
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda=lr_func, verbose=True)

step_count = 0
optim.zero_grad()
from tqdm import tqdm
for e in range(150):
  model.train()
  losses = []

  for  batch_data in tqdm(train_loader):
      img, label= batch_data["image"].to(device), batch_data["label"].to(device)
      # img = batch_data["image"].to(device)
      step_count += 1
      # img = img.to(device)
      # predicted_img, mask = model(img)

      loss, _, _ = model(img, mask_ratio=0.75)

      # loss = torch.mean((predicted_img - img) ** 2 * mask) / 0.75
      loss.backward()
      if step_count % 1 == 0:
          optim.step()
          optim.zero_grad()
      losses.append(loss.item())
  lr_scheduler.step()
  avg_loss = sum(losses) / len(losses)
  # writer.add_scalar('mae_loss', avg_loss, global_step=e)
  print(f'In epoch {e}, average traning loss is {avg_loss}.')

  ''' visualize the first 16 predicted images on val dataset'''
  model.eval()
  torch.save(model,'/content/drive/MyDrive/pretrained_with_oasis_inputlevelgauss.pt')

Adjusting learning rate of group 0 to 1.4062e-05.


100%|██████████| 160/160 [03:09<00:00,  1.18s/it]


Adjusting learning rate of group 0 to 2.8125e-05.
In epoch 0, average traning loss is 0.18276253589428962.


100%|██████████| 160/160 [03:10<00:00,  1.19s/it]


Adjusting learning rate of group 0 to 4.2187e-05.
In epoch 1, average traning loss is 0.15998981767334045.


...

Adjusting learning rate of group 0 to 6.1676e-08.
In epoch 147, average traning loss is 2.2041050942722683e-09.


100%|██████████| 160/160 [03:11<00:00,  1.20s/it]


Adjusting learning rate of group 0 to 1.5421e-08.
In epoch 148, average traning loss is 2.192056132443998e-09.


100%|██████████| 160/160 [03:11<00:00,  1.20s/it]


Adjusting learning rate of group 0 to 0.0000e+00.
In epoch 149, average traning loss is 2.1910369449318344e-09.
